# MFA using INCA in MATLAB - now all in Python!

This is an example notebook that makes use of functions that can write a MATLAB script that runs an MFA analysis using INCA. INCA is a MATLAB software. To make it easier for users, MATLAB can be run here in the notebook using an engine. The functions can be found in the "[INCA_script_generator.py](INCA_script_generator.py)" file. See the instructions below on how to use this notebook. It is assumed that your working directory is where this notebook is located, in the "examples" folder.

# MATLAB

Get a free academic licence and install MATLAB from https://www.mathworks.com. Then, install the engine API following the guide provided [under this link](https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html). In short, you will have to go to your MATLAB root folder (find your installation and open that folder) and go to "/extern/engines/python" and run "python setup.py install" from the command line.

# INCA

"INCA (Isotopomer Network Compartmental Analysis) is a MATLAB-based software package for isotopomer network modeling and metabolic flux analysis." You can read more about it in [Young, 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3998137/pdf/btu015.pdf).

You have to get a free academic licence for INCA from [the Vanderbilt University website](http://mfa.vueinnovations.com/licensing) (the second option is the relevant one) and install it. Note the path to the base directory of your INCA installation, you will need it later.

#### Import customized functions for INCA utilization

In [1]:
from BFAIR.INCA import INCA_script
import pandas as pd
import numpy as np
import time
import ast
import matlab.engine

#### Initialize the script

In [2]:
INCA_script = INCA_script()

#### Import the data

In [3]:
# measured fragments/MS data, tracers and measured fluxes should be limited to one experiment

atomMappingReactions_data_I = pd.read_csv('data/MFA_modelInputsData/data_stage02_isotopomer_atomMappingReactions2.csv')
modelReaction_data_I = pd.read_csv('data/MFA_modelInputsData/data_stage02_isotopomer_modelReactions.csv')
atomMappingMetabolite_data_I = pd.read_csv('data/MFA_modelInputsData/data_stage02_isotopomer_atomMappingMetabolites.csv')
measuredFluxes_data_I = pd.read_csv('data/MFA_modelInputsData/data_stage02_isotopomer_measuredFluxes.csv')
experimentalMS_data_I = pd.read_csv('data/MFA_modelInputsData/data-1604345289079.csv')
tracer_I = pd.read_csv('data/MFA_modelInputsData/data_stage02_isotopomer_tracers.csv')

#### Exclude data for irreleavnt experiments and models

In [4]:
# The files need to be limited by model id and mapping id, I picked "ecoli_RL2013_02" here
atomMappingReactions_data_I = INCA_script.limit_to_one_model(atomMappingReactions_data_I, 'mapping_id', 'ecoli_RL2013_02')
modelReaction_data_I = INCA_script.limit_to_one_model(modelReaction_data_I, 'model_id', 'ecoli_RL2013_02')
atomMappingMetabolite_data_I = INCA_script.limit_to_one_model(atomMappingMetabolite_data_I, 'mapping_id', 'ecoli_RL2013_02')
measuredFluxes_data_I = INCA_script.limit_to_one_model(measuredFluxes_data_I, 'model_id', 'ecoli_RL2013_02')

# Limiting fluxes, fragments and tracers to one experiment
measuredFluxes_data_I = INCA_script.limit_to_one_experiment(measuredFluxes_data_I, 'experiment_id', 'WTEColi_113C80_U13C20_01')
experimentalMS_data_I = INCA_script.limit_to_one_experiment(experimentalMS_data_I, 'experiment_id', 'WTEColi_113C80_U13C20_01')
tracer_I = INCA_script.limit_to_one_experiment(tracer_I, 'experiment_id', 'WTEColi_113C80_U13C20_01')

#### Generate the MATLAB script and save it in your working directory. The last argument in the script_generator function will name your future .mat file

In [5]:
script = INCA_script.script_generator(
    modelReaction_data_I,
    atomMappingReactions_data_I,
    atomMappingMetabolite_data_I,
    measuredFluxes_data_I,
    experimentalMS_data_I,
    tracer_I
)
INCA_script.save_INCA_script(script, "testscript")
runner = INCA_script.runner_script_generator('TestFile', 10)
print(runner)
INCA_script.save_runner_script(runner=runner, scriptname="testscript")

There is no stoichimetriy given for: ATPM
There is no stoichimetriy given for: Ec_Biomass_INCA
There is no stoichimetriy given for: EX_nh4_LPAREN_e_RPAREN_
There is no stoichimetriy given for: EX_o2_LPAREN_e_RPAREN_
There is no stoichimetriy given for: EX_so4_LPAREN_e_RPAREN_
There is no stoichimetriy given for: FADR_NADH_CYTBD_HYD_ATPS4r
There is no stoichimetriy given for: NADH_CYTBD_HYD_ATPS4r
There is no stoichimetriy given for: NADTRHD_THD2pp
There is no stoichimetriy given for: NADTRHD_THD2pp_reverse
f=estimate(m,10);

f=continuate(f,m);

filename = 'TestFile.mat';
save(filename,'f','m')


#### Provide the path to you INCA installation, your working directory and the name of the previously generated MATLAB script

In [6]:
INCA_base_directory = # ADD YOUR BASE DIRECTORY HERE, e.g. "/Users/Username/Documents/INCAv1.9"
script_folder = %pwd
matlab_script = "testscript"
runner_script = matlab_script + "_runner"

#### INCA will be started and your script run in MATLAB. This will produce the .mat file specified above

In [7]:
INCA_script.run_INCA_in_MATLAB(INCA_base_directory, script_folder, matlab_script, runner_script)

--- 270.8612148761749 seconds -
